# DataJoint Workflow Guide for Creating a New Ephys Session


Creating and inserting a new session in a DataJoint pipeline is crucial because sessions are the foundational unit of analysis, linking all collected data (e.g., neural signals, behavioral events) to a specific time period, thus enabling structured and comprehensive analysis of neuroscience experiments.

This notebook provides a walkthrough for adding a new ephys session to the pipeline.


**_Note:_**

- The examples in this notebook use a sample dataset. Replace these entries with your actual database entries to access and analyze your data.


### **Key Steps**


- **Setup**

- **Step 1: Select an Experiment from the List in the `Experiment` Table**

- **Step 2: Select Other Specific Information for the New Session**

- **Step 3: Create and Insert a New `Session` and `SessionProbe`**


#### **Setup**


First, import the necessary packages for the data pipeline and essential schemas.


In [1]:
## Ensure the correct working directory

import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime

In [3]:
from workflow.pipeline import culture, ephys, probe

[2024-10-31 18:00:55,669][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-10-31 18:00:57,248][INFO]: Connected milagros@db.datajoint.com:3306


#### **Step 1: Select an Experiment from the List in the `Experiment` Table**


The `Experiment` table contains the experiment to be performed on each organoid under different drug conditions.


In [4]:
culture.Experiment().proj(
    "experiment_end_time", "drug_name", "drug_concentration", "experiment_plan"
)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
E01,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E02,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E03,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E04,2024-08-30 19:43:00,2024-08-30 19:51:00,Control,nan,ephys
E09,2024-08-30 13:09:00,2024-08-30 13:13:00,Control,1.0,ephys
E10,2024-08-30 13:09:00,2024-08-30 13:13:00,Control,1.0,ephys
E11,2024-08-30 13:09:00,2024-08-30 13:13:00,Control,1.0,ephys
E12,2024-08-30 13:09:00,2024-08-30 13:13:00,Control,1.0,ephys
MB01,2024-08-30 19:00:00,2024-08-30 19:26:00,Control,nan,ephys
MB02,2024-08-30 19:00:00,2024-08-30 19:26:00,Control,nan,ephys


Let's choose one existing experiment for a organoid ID and a drug name:


In [71]:
exp_key = dict(organoid_id="MB07")

In [72]:
(culture.Experiment & exp_key).proj(
    "experiment_end_time", "drug_name", "drug_concentration", "experiment_plan"
)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
MB07,2024-09-07 14:49:00,2024-09-07 15:02:00,Control,nan,ephys


To define a single ephys session for the chosen experiment, the following information will be needed:

- the organoid ID.
- the start and end time.
- the `probe`, `port_id`, and `used_electrodes`.

Let's start defining each of these fields.


In [73]:
organoid_id, experiment_start_time = (culture.Experiment & exp_key).fetch1(
    "organoid_id", "experiment_start_time"
)

#### **Step 2: Select Other Specific Information for the New Session**


In [74]:
ephys.EphysSession.heading

# User defined ephys session for downstream analysis.
organoid_id          : varchar(4)                   # e.g. O17
experiment_start_time : datetime                     # 
insertion_number     : tinyint unsigned             # 
start_time           : datetime                     # 
end_time             : datetime                     # 
---
session_type=null    : enum('lfp','spike_sorting','both','test') # 

The `session_type` can be defined for `lfp` or `spike_sorting` analysis:


In [75]:
session_type = "spike_sorting"
insertion_number = 0

The objective of this notebook is to guide you through the process of inserting an ephys session with the `session_type` parameter set to `spike_sorting`. If there is already an entry in the table for this session type or others (such as `lfp`), you may consider updating the `session_type` to include both LFP and spike sorting analyses.


Let's define now the start and end times for the new session:


In [88]:
start_time = "2024-09-07 14:57:00"
end_time = "2024-09-07 14:02:00"

The duration for `lfp` sessions should not exceed **30 minutes**, and for `spike_sorting` sessions, it should not exceed **120 minutes**.


In [89]:
SPIKE_SORTING_DURATION = 120  # minutes
LFP_DURATION = 30  # minutes

# Start and end time of the session. It should be within the experiment time range
start_time_str = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
end_time_str = datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S")
duration = (end_time_str - start_time_str).total_seconds() / 60  # minutes

if session_type == "spike_sorting":
    assert (
        duration <= SPIKE_SORTING_DURATION
    ), f"Session type 'spike_sorting' duration must be less than {SPIKE_SORTING_DURATION} minutes"

elif session_type == "lfp":
    assert (
        duration <= LFP_DURATION
    ), f"Session type 'lfp' duration must be less than {LFP_DURATION} minutes"

In [90]:
new_session_key = dict(
    organoid_id=organoid_id,
    experiment_start_time=experiment_start_time,
    insertion_number=insertion_number,
    start_time=start_time,
    end_time=end_time,
)
new_session_key

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:57:00',
 'end_time': '2024-09-07 14:02:00'}

#### **Step 3: Create and Insert a New `Session` and `SessionProbe`**


Once all the session information has been selected, let's define the ephys session here:


In [91]:
session_info = dict(
    **new_session_key,
    session_type=session_type,
)

session_info

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:57:00',
 'end_time': '2024-09-07 14:02:00',
 'session_type': 'spike_sorting'}

Let's define now the session probe information:


In [80]:
ephys.Port()

port_id
A
B
C
D


In [81]:
probe.Probe()

probe unique identifier for this model of probe (e.g. serial number),probe_type e.g. A1x32-6mm-100-177-H32_21mm,probe_comment
12D4D,A1x32-6mm-100-177-H32_21mm,
12D51,A1x32-6mm-100-177-H32_21mm,
12D76,A1x32-6mm-100-177-H32_21mm,
12D77,A1x32-6mm-100-177-H32_21mm,
Q983,A1x32-6mm-100-177-H32_21mm,
T590,A1x32-6mm-100-177-H32_21mm,
T591,A1x32-6mm-100-177-H32_21mm,
T593,A1x32-6mm-100-177-H32_21mm,
T595,A1x32-6mm-100-177-H32_21mm,


In [92]:
session_probe_info = dict(
    **new_session_key,
    probe="Q983",  # probe serial number
    port_id="C",  # Port ID ("A", "B", etc.)
    used_electrodes=[],  # electrodes used for the session; empty if all electrodes were used
)

session_probe_info

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:57:00',
 'end_time': '2024-09-07 14:02:00',
 'probe': 'Q983',
 'port_id': 'C',
 'used_electrodes': []}

- `insert1` inserts a single record into a table.

- `insert` inserts a collection of rows.

- `skip_duplicates=True` prevents errors by skipping duplicate entries.


**Importance of skip_duplicates**:
DataJoint provides powerful tools for managing and enforcing **data integrity**. One such tool is the `skip_duplicates` parameter in `insert` and `insert1`:

- Unique Constraints: Ensures each entry is unique, avoiding redundant data.

- Error Handling: Prevents workflow interruption by skipping duplicate entries instead of raising errors.


When inserting data, if an entry with the same primary key already exists, attempting to insert it again without handling duplicates will raise a `DuplicateError`, interrupting the workflow.


In [95]:
session_info

{'organoid_id': 'MB07',
 'experiment_start_time': datetime.datetime(2024, 9, 7, 14, 49),
 'insertion_number': 0,
 'start_time': '2024-09-07 14:57:00',
 'end_time': '2024-09-07 14:02:00',
 'session_type': 'spike_sorting'}

In [97]:
# Expected to receive a `DuplicateError` here if the session_info is already in the table
ephys.EphysSession.insert1(
    {
        "organoid_id": "MB07",
        "experiment_start_time": "2024-09-07 14:49:00",
        "insertion_number": 0,
        "start_time": "2024-09-07 14:57:00",
        "end_time": "2024-09-07 14:02:00",
    }
)

Let's confirm that the session exists already in the table:


In [98]:
ephys.EphysSession & new_session_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
MB07,2024-09-07 14:49:00,0,2024-09-07 14:57:00,2024-09-07 14:02:00,None


In [99]:
ephys.EphysSessionProbe.insert1(session_probe_info)

In [100]:
ephys.EphysSessionProbe & new_session_key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
MB07,2024-09-07 14:49:00,0,2024-09-07 14:57:00,2024-09-07 14:02:00,Q983,C,=BLOB=


In [ ]:
ephys.EphysSessionInfo & new_session_key

To review all the ephys sessions inserted for `spike_sorting`:


In [ ]:
(ephys.EphysSession & "session_type = 'spike_sorting'")